# Libraries

In [8]:
# Import libraries

from os import listdir
import string
import pandas as pd
import numpy as np  
import pandas as pd 
from IPython.display import display, clear_output
import re           
from bs4 import BeautifulSoup 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
import re, string, unicodedata
import nltk
import pickle
import inflect
from bs4 import BeautifulSoup
import tensorflow as tf

import transformers

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 

pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [9]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Please install GPU version of TF


# Load data

In [10]:
data = pd.read_csv("../../preprocessing/test_dataset.csv")

In [11]:
data["cleaned_highlight"] = data["cleaned_highlight"].str.replace("starttoken", "")
data["cleaned_highlight"] = data["cleaned_highlight"].str.replace("endtoken", "")


In [12]:
stories = data["cleaned_text"]
summaries = data["cleaned_highlight"]

# Model

In [13]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, BartForConditionalGeneration,  BartTokenizer, PegasusForConditionalGeneration, PegasusTokenizer

# initialize the model architecture and weights
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-base")
model_bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# initialize the model tokenizer
tokenizer_t5 = T5Tokenizer.from_pretrained("t5-base")
tokenizer_bart = BartTokenizer.from_pretrained("facebook/bart-base")



Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Inference

In [14]:
original_text = []
original_summary = []
created_summary_t5 = []
created_summary_bart = []
created_summary_pegasus = []

for i in range(0,2000):
    clear_output(wait=True)
    print(i)
    original_text.append(stories[i])
    original_summary.append(summaries[i])
    #t5
    inputs_t5 = tokenizer_t5.encode("summarize: " + stories[i], return_tensors="pt", max_length=300, truncation=True)
    outputs_t5 = model_t5.generate(
        inputs_t5, 
        max_length=12, 
        min_length=3, 
        length_penalty=1.0,  
        early_stopping=True)
    created_summary_t5.append(tokenizer_t5.decode(outputs_t5[0]))
    #bart
    inputs_bart = tokenizer_bart.encode(stories[i], return_tensors="pt", max_length=300, truncation=True)
    outputs_bart = model_bart.generate(
        inputs_bart, 
        max_length=12, 
        min_length=3, 
        length_penalty=1.0,  
        early_stopping=True)
    created_summary_bart.append(tokenizer_bart.decode(outputs_bart[0]))
    

1999


KeyError: 1999

In [22]:
results = pd.DataFrame()
results["Original_text"] = original_text
results["Original_summary"] = original_summary
results["Created_summary_t5"] = created_summary_t5
results["Created_summary_bart"] = created_summary_bart
results["Created_summary_t5"] = results["Created_summary_t5"].str.replace("<pad> ", "")
results["Created_summary_bart"] = results["Created_summary_bart"].str.replace("</s><s>", "")
results["Created_summary_bart"] = results["Created_summary_bart"].str.replace("</s>", "")


In [23]:
results

,Original_text,Original_summary,Created_summary_t5,Created_summary_bart
0,united states president barack obama paved way imposing economic sanctions russia however would ineffective naive response crisis ukraine economic sanctions little independent usefulness pursuit n...,robert pape argues economic sanctions russia would work,russia would ineffective ukraine economic sanctions,united states president barack obama paved way
1,yes read headline right proof marilyn monroe sexiest woman ever walk earth years death people still willing spend money facial x rays auctioned julien auctions according vanity fair six facial x r...,six facial x rays marilyn monroe surfaced,marilyn monroe x ray,yes read headline right proof marilyn monroe
2,anarchist social worker raided feds wants computers manuscripts pick axes back argues authorities violated constitution rights mentally ill clients searching evidence broke anti rioting law twitte...,elliott madison home raided fbi october,anarchist social worker raided feds house,anarchist social worker raided feds wants computers
3,cast marvel avengers age ultron keeps getting better sunday thor co star idris elba let slip interview uk telegraph tom hiddleston appear next may avengers sequel elba played asgard guardian heimd...,idris elba confirmed avengers age ultron,idris elba says,cast marvel avengers age ultron keeps getting
4,british world champion amir khan retained world boxing association light welterweight title saturday night clinical stopppage victory challenger paulie malignaggi madison square garden new york ye...,british world champion amir khan retains wba light welterweight title,amir khan retains light welt,british world champion amir khan
...,...,...,...,...
1994,former president george w bush briefly jumped back national conversation tuesday night interview tonight show bush told host jay leno miss spotlight thoroughly enjoying post presidential life wedn...,george w bush goes tonight show bill clinton gets medal freedom,former president george w bush briefly jumped,former president george w bush briefly jumped back
1995,new york governors massachusetts new hampshire declared states emergency friday high winds rain snow swept northeast leaving least million homes businesses without power ireporter candy novoa says...,new least million without power across four northeastern states,new hampshire declares states emergency.,new york governors massachusetts new hamp
1996,port au prince haiti like many people done time haiti gary garner needs good cry past five days salt lake city utah physician held dying man arms amputated fingers toes remember needs rest friday ...,doctor utah looks way home days volunteering,gary garner said low voice quietly gently suffering showing,port au prince haiti like many people done
1997,paul schaefer former nazi fled germany founded cult like commune chile died saturday prison hospital schaefer serving year sentence national penitentiary santiago sexually abusing children notorio...,former nazi officer paul schaefer dies chile cardiopulmonary arrest,paul schaefer dies in,paul schaefer former nazi fled


In [24]:
results.to_csv("results_predictions_transformers.csv")

In [25]:
results=pd.read_csv("results_predictions_transformers.csv")

In [26]:
reference_sentences = results["Original_summary"].to_list()
summary_sentences_t5 = results["Created_summary_t5"].to_list()
summary_sentences_bart = results["Created_summary_bart"].to_list()


# Evaluation

In [27]:
from rouge import rouge_n_sentence_level
from rouge import rouge_l_sentence_level
from rouge import rouge_n_summary_level
from rouge import rouge_l_summary_level
from rouge import rouge_w_sentence_level
from rouge import rouge_w_summary_level
import statistics

mean_rouge_r2_t5 = 0
mean_rouge_r1_t5= 0
mean_rouge_r2_bart= 0
mean_rouge_r1_bart= 0

summary_sentences_list = [summary_sentences_t5, summary_sentences_bart]
mean_rouge_r2_list = [mean_rouge_r2_t5, mean_rouge_r2_bart]
mean_rouge_r1_list = [mean_rouge_r1_t5, mean_rouge_r1_bart]

for iteration in range(2):
    print("Model: ", iteration)
    
    list_rouge_r2 = []
    list_recall_r2 = []
    list_precision_r2 = []
    list_rouge_r1= []
    list_recall_r1 = []
    list_precision_r1 = []

    for i in range(0, len(reference_sentences)):
        clear_output(wait=True)
        print(i)
        reference_sentence = reference_sentences[i].split()
        summary_sentence = summary_sentences_list[iteration][i].split()
        
        # Calculate ROUGE-2.
        recall_r2, precision_r2, rouge_r2 = rouge_n_sentence_level(summary_sentence, reference_sentence, 2)

        list_rouge_r2.append(rouge_r2)
        list_recall_r2.append(recall_r2)
        list_precision_r2.append(precision_r2)

        # Calculate ROUGE-1.
        recall_r1, precision_r1, rouge_r1 = rouge_n_sentence_level(summary_sentence, reference_sentence, 1)

        list_rouge_r1.append(rouge_r1)
        list_recall_r1.append(recall_r1)
        list_precision_r1.append(precision_r1)


    mean_rouge_r2_list[iteration] = statistics.mean(list_rouge_r2)  
    mean_rouge_r1_list[iteration] = statistics.mean(list_rouge_r1)  

    print("Mean ROUGE-2 : ", mean_rouge_r2_list[iteration])
    print("Mean ROUGE-1 : ", mean_rouge_r1_list[iteration])

1998
Mean ROUGE-2 :  0.07226925170227316
Mean ROUGE-1 :  0.17462091734026464


In [31]:
print("Model T5")
print("Rouge 2: ", mean_rouge_r2_list[0], "Rouge 1: ",mean_rouge_r1_list[0])
print("Model BART")
print("Rouge 2: ", mean_rouge_r2_list[1], "Rouge 1: ",mean_rouge_r1_list[1])

Model T5
Rouge 2:  0.08805643454159663 Rouge 1:  0.20506368791354937
Model BART
Rouge 2:  0.07226925170227316 Rouge 1:  0.17462091734026464
